In [1]:
import utils as utils
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from LTransform import Transform

In [2]:
data_path = "./000128/sub-Jenkins/sub-Jenkins_ses-full_desc-train_behavior+ecephys.nwb"
D, trial_info, units = utils.read_mc_maze(data_path)
tools = utils.Analysis_tools(fs = 1000)


In [ ]:
condition_columns = ['trial_type', 'trial_version']
align_column = 'move_onset_time'
#unit = [2561, 1011]
unit = units.index.values
t_range = [-100, 500]


data_aligned = tools.align_fr(trial_info, units, condition_columns, align_column, unit, t_range)

cmap = sns.color_palette('muted', n_colors=108)
for i in range(data_aligned.shape[0]):
    plt.plot(data_aligned[i, :, 1])


In [ ]:
which_unit = 2561
t = [1, 2]

t_pad = 0.5 # Seconds
dt = 1/1000

plt.subplot(2,1,1)
stem = tools.get_stem(units, which_unit, t, plot=True, dt=dt, t_pad=t_pad)
plt.subplot(2,1,2)
fr = tools.get_fr(which_unit, units, t, plot=True, dt=dt, t_pad=t_pad)

In [ ]:
condition_columns = ['trial_type', 'trial_version']
align_column = 'move_onset_time'
channel_column = 'hand_pos'
t_range = [-50, 450]

data_aligned = tools.align_continuous(trial_info, D, condition_columns, align_column, channel_column, t_range)

cmap = sns.color_palette('muted', n_colors=108)
for i in range(data_aligned.shape[0]):
    plt.plot(data_aligned[i, :, 0], data_aligned[i, :, 1], color=cmap[i])


In [16]:
condition_columns = ['trial_type', 'trial_version']
align_column = 'move_onset_time'
channel_column = 'hand_vel'
t_range = [-50, 400]

data_aligned = tools.align_continuous(trial_info, D, condition_columns, align_column, channel_column, t_range)

cmap = sns.color_palette('muted', n_colors=108)
for i in range(data_aligned.shape[0]):
    plt.plot(np.sqrt(data_aligned[i, :, 0]**2 + data_aligned[i, :, 1]**2), color=cmap[i])


In [22]:
condition_columns = ['trial_type', 'trial_version']
align_column = 'move_onset_time'
unit = [2561, 1011]
unit = units.index.values
unit = units.loc[units['location'] == 'M1'].index.values
t_range = [-50, 600]


data_aligned = tools.align_fr(trial_info, units, condition_columns, align_column, unit, t_range)

cmap = sns.color_palette('muted', n_colors=108)
for i in range(data_aligned.shape[0]):
    plt.plot(data_aligned[i, :, 4])


The process has forked and you cannot use this CoreFoundation functionality safely. You MUST exec().
Break on __THE_PROCESS_HAS_FORKED_AND_YOU_CANNOT_USE_THIS_COREFOUNDATION_FUNCTIONALITY___YOU_MUST_EXEC__() to debug.
The process has forked and you cannot use this CoreFoundation functionality safely. You MUST exec().
Break on __THE_PROCESS_HAS_FORKED_AND_YOU_CANNOT_USE_THIS_COREFOUNDATION_FUNCTIONALITY___YOU_MUST_EXEC__() to debug.
The process has forked and you cannot use this CoreFoundation functionality safely. You MUST exec().
Break on __THE_PROCESS_HAS_FORKED_AND_YOU_CANNOT_USE_THIS_COREFOUNDATION_FUNCTIONALITY___YOU_MUST_EXEC__() to debug.
The process has forked and you cannot use this CoreFoundation functionality safely. You MUST exec().
Break on __THE_PROCESS_HAS_FORKED_AND_YOU_CANNOT_USE_THIS_COREFOUNDATION_FUNCTIONALITY___YOU_MUST_EXEC__() to debug.
The process has forked and you cannot use this CoreFoundation functionality safely. You MUST exec().
Break on __THE_PROCESS_HAS_

In [23]:
%matplotlib qt
rate_stack = np.vstack(data_aligned)
rate_scaled = (rate_stack - np.mean(rate_stack, axis=0))/((np.max(rate_stack, axis=0) - np.min(rate_stack, axis=0)) + 5)
rate_scaled = rate_scaled.reshape((data_aligned.shape[0], data_aligned.shape[1], -1))

transform = Transform(num_latent=3)
transform.fit(rate_scaled, method='FA')
X_ld = transform.transform(rate_scaled, ensure_orthogonality=True)

cmap = sns.color_palette('muted', n_colors=108)

fig = plt.figure(figsize=(9, 6))
ax = fig.add_subplot(111, projection='3d')
for c in range(108):
    ax.plot(X_ld[c, :, 0], X_ld[c,:, 1], X_ld[c,:, 2], lw=2, c=cmap[c])
    ax.scatter(X_ld[c,0, 0], X_ld[c,0, 1], X_ld[c,0, 2], color=cmap[c])

Found tensor- Reshaping data to -1, num_unit
Found tensor- Reshaping data to -1, num_unit


In [24]:
rate_stack = np.vstack(data_aligned)
# Normalization
#rate_scaled = (rate_stack - np.mean(rate_stack, axis=0))/np.std(rate_stack, axis=0)
# Soft Normalization
rate_scaled = (rate_stack - np.mean(rate_stack, axis=0))/((np.max(rate_stack, axis=0) - np.min(rate_stack, axis=0)) + 5)


from sklearn.decomposition import PCA
pca = PCA(n_components=3)
traj_stack = pca.fit_transform(rate_scaled)


X_ld = traj_stack.reshape((data_aligned.shape[0], data_aligned.shape[1], -1))


cmap = sns.color_palette('muted', n_colors=108)

fig = plt.figure(figsize=(9, 6))
ax = fig.add_subplot(111, projection='3d')
for c in range(108):
    ax.plot(X_ld[c, :, 0], X_ld[c,:, 1], X_ld[c,:, 2], lw=2, c=cmap[c])
    ax.scatter(X_ld[c,0, 0], X_ld[c,0, 1], X_ld[c,0, 2], color=cmap[c])

In [25]:
rate_stack = np.mean(data_aligned, axis=0)
# Normalization
#rate_scaled = (rate_stack - np.mean(rate_stack, axis=0))/np.std(rate_stack, axis=0)
# Soft Normalization
rate_scaled = (rate_stack - np.mean(rate_stack, axis=0))/((np.max(rate_stack, axis=0) - np.min(rate_stack, axis=0)) + 5)




pca = PCA(n_components=3)
traj_stack = pca.fit_transform(rate_scaled)


rate_stack = np.vstack(data_aligned)
rate_scaled = (rate_stack - np.mean(rate_stack, axis=0))/((np.max(rate_stack, axis=0) - np.min(rate_stack, axis=0)) + 5)


traj_stack = pca.transform(rate_scaled)
X_ld = traj_stack.reshape((data_aligned.shape[0], data_aligned.shape[1], -1))


cmap = sns.color_palette('muted', n_colors=108)

fig = plt.figure(figsize=(9, 6))
ax = fig.add_subplot(111, projection='3d')
for c in range(108):
    ax.plot(X_ld[c, :, 0], X_ld[c,:, 1], X_ld[c,:, 2], lw=2, c=cmap[c])
    ax.scatter(X_ld[c,0, 0], X_ld[c,0, 1], X_ld[c,0, 2], color=cmap[c])

plt.show()

for c in range(108):
    plt.plot(X_ld[c, :, 0], X_ld[c,:, 1],c=cmap[c])

In [26]:
for c in range(108):
    plt.plot(X_ld[c, :, 0], X_ld[c,:, 1],c=cmap[c])
    plt.show()